In [ ]:
from pathlib import Path
import pandas as pd

def resolve_data_dir() -> Path:
    candidates = [
        Path("data"),
        Path("01-docker-sql-trfm") / "data",
        Path("..") / "data",
    ]
    for candidate in candidates:
        if candidate.exists():
            return candidate
    raise FileNotFoundError("Could not find data directory.")

DATA_DIR = resolve_data_dir()
df = pd.read_parquet(DATA_DIR / "green_tripdata_2025-11.parquet")

In [3]:
df.head

<bound method NDFrame.head of        VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
0             2  2025-11-01 00:34:48   2025-11-01 00:41:39                  N   
1             2  2025-11-01 00:18:52   2025-11-01 00:24:27                  N   
2             2  2025-11-01 01:03:14   2025-11-01 01:15:24                  N   
3             2  2025-11-01 00:10:57   2025-11-01 00:24:53                  N   
4             1  2025-11-01 00:03:48   2025-11-01 00:19:38                  N   
...         ...                  ...                   ...                ...   
46907         2  2025-11-30 19:58:34   2025-11-30 20:14:28                NaN   
46908         2  2025-11-30 19:34:00   2025-11-30 19:46:00                NaN   
46909         2  2025-11-30 21:46:46   2025-11-30 22:17:55                NaN   
46910         2  2025-11-30 21:00:00   2025-11-30 21:15:00                NaN   
46911         2  2025-11-30 23:26:00   2025-11-30 23:42:00                NaN  

In [ ]:
df_zones = pd.read_csv(DATA_DIR / "taxi_zone_lookup.csv")
df_zones.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [5]:
import duckdb

In [4]:
duckdb.sql("""
SELECT COUNT (*) AS short_trips
FROM df
WHERE lpep_pickup_datetime >= '2025-11-01'
    AND lpep_pickup_datetime < '2025-12-01'
    AND trip_distance <= 1
""").show()

┌─────────────┐
│ short_trips │
│    int64    │
├─────────────┤
│        8007 │
└─────────────┘



In [5]:
duckdb.sql("""
SELECT
  MIN(lpep_pickup_datetime) AS min_pickup,
  MAX(lpep_pickup_datetime) AS max_pickup,
  COUNT(*) AS total_rows,
  COUNT(*) FILTER (WHERE trip_distance IS NULL) AS null_trip_distance,
  COUNT(*) FILTER (WHERE trip_distance <= 1) AS short_trips_raw
FROM df
""").show()

┌─────────────────────┬─────────────────────┬────────────┬────────────────────┬─────────────────┐
│     min_pickup      │     max_pickup      │ total_rows │ null_trip_distance │ short_trips_raw │
│      timestamp      │      timestamp      │   int64    │       int64        │      int64      │
├─────────────────────┼─────────────────────┼────────────┼────────────────────┼─────────────────┤
│ 2025-10-26 20:23:16 │ 2025-12-01 20:29:00 │      46912 │                  0 │            8009 │
└─────────────────────┴─────────────────────┴────────────┴────────────────────┴─────────────────┘



In [12]:
duckdb.sql("""
SELECT
	CAST(lpep_pickup_datetime AS DATE) AS pickup_day,
	trip_distance
FROM df
WHERE trip_distance < 100
	AND lpep_pickup_datetime >= '2025-11-01'
	AND lpep_pickup_datetime < '2025-12-01'
ORDER BY trip_distance DESC, lpep_pickup_datetime ASC
LIMIT 10
""").show()

┌────────────┬───────────────┐
│ pickup_day │ trip_distance │
│    date    │    double     │
├────────────┼───────────────┤
│ 2025-11-14 │         88.03 │
│ 2025-11-20 │         73.84 │
│ 2025-11-23 │         45.26 │
│ 2025-11-22 │         40.16 │
│ 2025-11-15 │         39.81 │
│ 2025-11-22 │         39.47 │
│ 2025-11-11 │          39.0 │
│ 2025-11-19 │         38.68 │
│ 2025-11-16 │         37.67 │
│ 2025-11-10 │         36.46 │
├────────────┴───────────────┤
│ 10 rows          2 columns │
└────────────────────────────┘



In [14]:
duckdb.sql("""
SELECT
    DATE(lpep_pickup_datetime) AS pickup_day,
    MAX(trip_distance) AS max_trip_distance
FROM df
WHERE lpep_pickup_datetime >= '2025-11-01'
  AND lpep_pickup_datetime < '2025-12-01'
  AND trip_distance < 100
GROUP BY pickup_day
ORDER BY max_trip_distance DESC
LIMIT 10
""").show()

┌────────────┬───────────────────┐
│ pickup_day │ max_trip_distance │
│    date    │      double       │
├────────────┼───────────────────┤
│ 2025-11-14 │             88.03 │
│ 2025-11-20 │             73.84 │
│ 2025-11-23 │             45.26 │
│ 2025-11-22 │             40.16 │
│ 2025-11-15 │             39.81 │
│ 2025-11-11 │              39.0 │
│ 2025-11-19 │             38.68 │
│ 2025-11-16 │             37.67 │
│ 2025-11-10 │             36.46 │
│ 2025-11-26 │              36.1 │
├────────────┴───────────────────┤
│ 10 rows              2 columns │
└────────────────────────────────┘



In [7]:
duckdb.sql("""
SELECT
    z."Zone" AS pickup_zone,
    SUM(g.total_amount) AS total_revenue
FROM df as g
JOIN df_zones as z
    ON g."PULocationID" = z."LocationID"
WHERE g.lpep_pickup_datetime >= '2025-11-18'
    AND g.lpep_pickup_datetime < '2025-11-19'
GROUP BY z."Zone"
ORDER BY total_revenue DESC
LIMIT 5
""").show()

┌──────────────────────────┬────────────────────┐
│       pickup_zone        │   total_revenue    │
│         varchar          │       double       │
├──────────────────────────┼────────────────────┤
│ East Harlem North        │  9281.919999999998 │
│ East Harlem South        │  6696.130000000003 │
│ Central Park             │ 2378.7899999999995 │
│ Washington Heights South │            2139.05 │
│ Morningside Heights      │ 2100.5900000000006 │
└──────────────────────────┴────────────────────┘



In [8]:
duckdb.sql("""
SELECT
    zp."Zone" AS pickup_zone,
    zd."Zone" AS dropoff_zone,
    g.tip_amount AS tip
FROM df g
JOIN df_zones zp
    ON g."PULocationID" = zp."LocationID"
JOIN df_zones zd
    ON g."DOLocationID" = zd."LocationID"
WHERE g.lpep_pickup_datetime >= '2025-11-01'
    AND g.lpep_pickup_datetime < '2025-12-01'
    AND zp."Zone" = 'East Harlem North'
ORDER BY tip DESC
LIMIT 10
""").show()

┌───────────────────┬───────────────────────────────┬────────┐
│    pickup_zone    │         dropoff_zone          │  tip   │
│      varchar      │            varchar            │ double │
├───────────────────┼───────────────────────────────┼────────┤
│ East Harlem North │ Yorkville West                │  81.89 │
│ East Harlem North │ LaGuardia Airport             │   50.0 │
│ East Harlem North │ East Harlem North             │   45.0 │
│ East Harlem North │ Long Island City/Queens Plaza │  34.25 │
│ East Harlem North │ Outside of NYC                │   28.9 │
│ East Harlem North │ East Harlem North             │   26.0 │
│ East Harlem North │ JFK Airport                   │  23.53 │
│ East Harlem North │ Morningside Heights           │   20.0 │
│ East Harlem North │ East Harlem South             │   20.0 │
│ East Harlem North │ East Harlem South             │   20.0 │
├───────────────────┴───────────────────────────────┴────────┤
│ 10 rows                                          3 co